# Driving Towards a Brighter Future: 
## EV Adaption Rates in California and its Effect on PM 2.5 Emissions



In [1]:
# install pip if not already downloaded
# pip install openpyxl

In [2]:
# import
import pandas as pd 
import os
from urllib.request import urlretrieve
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
from scipy.stats import zscore

### Primary Dataset: Cleaning and Manipulation 
Our primary datasource was the California Vehicle Population dataset, which contains information on number of vehicles registered in each count or zip code by make, model, and fuel type. This file was uploaded to Github and stored in vehicle_data folder. Manually inspecting the vehicle data reveals there are three work sheets, where 'County' shows the registered cars per each county in California. Since we are studying the effects by county, this is the dataset we are interested in. 

In [3]:
# Let's create df from csv file 
countyvehicle_df = pd.read_excel('vehicle_data/california_vehicle.xlsx',sheet_name="County")

In [4]:
## We've isolated the worksheet, now let's take a look at the shape ##
countyvehicle_df.shape

(33542, 7)

In [5]:
## We see there are 33542 rows and 7 columns, now let's see what that looks like ##
countyvehicle_df.head()

,Data Year,County,Dashboard Fuel Type Group,Fuel Type,Make,Model,Number of Vehicles
0,2010,Alameda,Battery Electric (BEV),Battery Electric (BEV),Ford,Ranger,3
1,2010,Alameda,Battery Electric (BEV),Battery Electric (BEV),Tesla,Roadster,17
2,2010,Alameda,Diesel,Diesel,NaN,NaN,10939
3,2010,Alameda,Gasoline,Flex Fuel,NaN,NaN,10974
4,2010,Alameda,Gasoline,Gasoline,NaN,NaN,840577


In [6]:
## We're not interested in the Make or Model of the car, so let's drop those values ##
countyvehicle_df = countyvehicle_df.drop(columns=['Make','Model'])
countyvehicle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33542 entries, 0 to 33541
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Data Year                  33542 non-null  int64 
 1   County                     33542 non-null  object
 2   Dashboard Fuel Type Group  33542 non-null  object
 3   Fuel Type                  33542 non-null  object
 4   Number of Vehicles         33542 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.3+ MB


In [7]:
# Let's see how many unique Counties there are in California
countyvehicle_df['County'].unique()

array(['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
       'Contra Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn',
       'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake', 'Lassen',
       'Los Angeles', 'Madera', 'Marin', 'Mariposa', 'Mendocino',
       'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa', 'Nevada', 'Orange',
       'Out Of State', 'Placer', 'Plumas', 'Riverside', 'Sacramento',
       'San Benito', 'San Bernardino', 'San Diego', 'San Francisco',
       'San Joaquin', 'San Luis Obispo', 'San Mateo', 'Santa Barbara',
       'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou',
       'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Tehama', 'Trinity',
       'Tulare', 'Tuolumne', 'Ventura', 'Yolo', 'Yuba', 'Out of State'],
      dtype=object)

In [8]:
# There are vehicles present that are from out of state. Let's see what years have vehicles from out of state 
for year in countyvehicle_df['Data Year'].unique():
    out_of_state_2023 = countyvehicle_df[(countyvehicle_df['Data Year'] == year) & 
                                         (countyvehicle_df['County'] == 'Out of State')]
    
    total_out_of_state_2023 = out_of_state_2023['Number of Vehicles'].sum()
    if total_out_of_state_2023 != 0:
        print(f"The total number of vehicles for 'Out of State' in {year} is: {total_out_of_state_2023}")

The total number of vehicles for 'Out of State' in 2021 is: 403531
The total number of vehicles for 'Out of State' in 2022 is: 409644


In [9]:
# Since out of state is only present for two years (2021-2022), we'll drop those values and note our assumption that they have a marginal impact. 
calcountyvehicle_df = countyvehicle_df[~countyvehicle_df['County'].isin(['Out of State'])]

In [10]:
# Next, let's take a look at the fuel types that are used. We're interested in impact of EVs and non-fossil fuel based cars. 
# There are two columns that cover the fuel type, 'Dashboard Fuel Type Group' and 'Fuel Type' that are fully populated (length 33542). 
# Let's take a look at the difference between the two.
fuel_types_1 = calcountyvehicle_df['Dashboard Fuel Type Group'].unique()
fuel_types_2 = calcountyvehicle_df['Fuel Type'].unique()

fuel_types_df = pd.DataFrame({
    'Fuel Type': pd.Series(fuel_types_2),
    'Dashboard Fuel Type Group': pd.Series(fuel_types_1)
})

fuel_types_df


,Fuel Type,Dashboard Fuel Type Group
0,Battery Electric (BEV),Battery Electric (BEV)
1,Diesel,Diesel
2,Flex Fuel,Gasoline
3,Gasoline,Gasoline Hybrid
4,Gasoline Hybrid,Other
5,Natural Gas,Fuel Cell (FCEV)
6,Propane,Plug-in Hybrid (PHEV)
7,Fuel Cell (FCEV),NaN
8,Plug-in Hybrid (PHEV),NaN


# Brett, we can uncomment this if it's needed :D 

In [11]:


# # It appears Dashboard Fuel Type Group goes ahead and clusters them together. 
# # We're going to make our own the assumption and group together vehicles based on fuel type: 
# # Diesel, Flex Fuel, Gasoline, Gasoline Hybrid, Natural Gas, and Propane will be treated as Fossil Fuels, while
# # Battery Electric, Fuel Cell, and Plug-in Hybrid will be treated as EVs. 
# fossil_fuels = ['Diesel', 'Flex Fuel', 'Gasoline', 'Gasoline Hybrid', 'Natural Gas', 'Propane']
# evs = ['Battery Electric (BEV)', 'Fuel Cell (FCEV)', 'Plug-in Hybrid (PHEV)']
# def categorize_fuel(fuel_type):
#     if fuel_type in fossil_fuels:
#         return 'Fossil Fuel'
#     elif fuel_type in evs:
#         return 'EV'
#     else:
#         return 'Other'
# calcountyvehicle_df.loc[:, 'Fuel Category'] = calcountyvehicle_df['Fuel Type'].apply(categorize_fuel)
# # Print the DataFrame to check the new column
# calcountyvehicle_df.head()

In [12]:
# # Let's total the Number of Vehicles by Data Year, County, and Fuel Category. 
# fueltype_per_countyyear_totals_df = calcountyvehicle_df.groupby(['Data Year','County','Fuel Category'])['Number of Vehicles'].sum().reset_index()
# fueltype_per_countyyear_totals_df

In [13]:
# # Let's make a more sophisticated dataframe, so that each County has a EV and Fossil Fuel Category. This will allow us to more directly compare Counties and they Car Registration data. 
# for year in fueltype_per_countyyear_totals_df['Data Year'].unique():
#     for county in fueltype_per_countyyear_totals_df['County'].unique():
#         # Get the subset of the dataframe for this year and county
#         subset = fueltype_per_countyyear_totals_df[(fueltype_per_countyyear_totals_df['Data Year'] == year) & 
#                                                    (fueltype_per_countyyear_totals_df['County'] == county)]
#         
#         # Check if 'EV' exists in the 'Fuel Category' for this year and county
#         if 'EV' not in subset['Fuel Category'].values:
#             # If not, create a new row with 0 vehicles for 'EV'
#             new_row_ev = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['EV'], 'Number of Vehicles': [0]})
#             fueltype_per_countyyear_totals_df = pd.concat([fueltype_per_countyyear_totals_df, new_row_ev], ignore_index=True)
# 
#         # Check if 'Fossil Fuel' exists in the 'Fuel Category' for this year and county
#         if 'Fossil Fuel' not in subset['Fuel Category'].values:
#             # If not, create a new row with 0 vehicles for 'Fossil Fuel'
#             new_row_fossil = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['Fossil Fuel'], 'Number of Vehicles': [0]})
#             fueltype_per_countyyear_totals_df = pd.concat([fueltype_per_countyyear_totals_df, new_row_fossil], ignore_index=True)
# 
# # Optionally, sort the dataframe by 'Data Year', 'County', and 'Fuel Category' for better readability
# fueltype_per_countyyear_totals_df = fueltype_per_countyyear_totals_df.sort_values(by=['Data Year', 'County', 'Fuel Category']).reset_index(drop=True)
# 
# fueltype_per_countyyear_totals_df.head()

# Brett, I think this is what we actually need, methinks. Is this right? 

In [14]:
# Now that we know the Fuel Type categories, let's organize our dataset so that each County has a EV and Fossil Fuel Category. This will allow us to directly compare Counties and they Car Registration data. 
def categorize_fuel(fuel_type):
    """
    Categorize fuel types into 'Fossil Fuel', 'EV', or 'Other'.
    
    Args:
        fuel_type (str): The fuel type to categorize.
    
    Returns:
        str: The category of the fuel type.
    """
    fossil_fuels = ['Diesel', 'Flex Fuel', 'Gasoline', 'Gasoline Hybrid', 'Natural Gas', 'Propane']
    evs = ['Battery Electric (BEV)', 'Fuel Cell (FCEV)', 'Plug-in Hybrid (PHEV)']
    
    if fuel_type in fossil_fuels:
        return 'Fossil Fuel'
    elif fuel_type in evs:
        return 'EV'
    else:
        return 'Other'
def download_file(url, local_filename):
    """
    Download a file from a URL to a local filename.
    
    Args:
        url (str): The URL of the file to download.
        local_filename (str): The local file path to save the downloaded file.
    """
    try:
        urlretrieve(url, local_filename)
        print(f"File downloaded successfully and saved as {local_filename}.")
    except Exception as e:
        raise Exception(f"An error occurred while downloading the file: {e}")
    
def fueltype_per_countyyear_totals_df(excel_file):
    """
    Generate a DataFrame summarizing the number of vehicles by year, county, and fuel category.
    
    Args:
        excel_file (str): Path to the Excel file containing vehicle data.
    
    Returns:
        pd.DataFrame: A DataFrame with totals of vehicles by year, county, and fuel category.
    
    Raises:
        FileNotFoundError: If the specified Excel file is not found.
        ValueError: If the sheet 'County' is not found in the Excel file or if there are issues with the data format.
        """
    excel_file = os.path.join(os.getcwd(), excel_file)

    if not os.path.exists(excel_file):
        print(f"File {excel_file} not found locally. Attempting to download...")
        URL = 'https://www.energy.ca.gov/filebrowser/download/6311?fid=6311#block-symsoft-page-title'
        download_file(URL, excel_file)
    
    # Pull County Sheet from excel file ##
    try:
        # Read the Excel file
        countyvehicle_df = pd.read_excel(excel_file, sheet_name="County")
    except FileNotFoundError:
        raise FileNotFoundError(f"The file {excel_file} was not found.")
    except ValueError as e:
        raise ValueError("An error occurred while reading the Excel file. Ensure the file contains a sheet named 'County'.") from e
    except Exception as e:
        raise Exception("An unexpected error occurred while reading the Excel file.") from e
    
    ## Drop Irrelevent Columns ##
    try:
        countyvehicle_df = countyvehicle_df.drop(columns=['Make', 'Model'])
    except KeyError as e:
        raise KeyError("The specified columns to drop do not exist in the DataFrame.") from e
    
    
    ## Remove unwanted County Information ##
    cal_countyvehicle_df = countyvehicle_df[~countyvehicle_df['County'].isin(['Out of State'])]
    
    ## Simply fuel types and categories ##
    cal_countyvehicle_df.loc[:, 'Fuel Category'] = cal_countyvehicle_df['Fuel Type'].apply(categorize_fuel)
    
    ## Create final data frame based on year, county, fuel category, and number of vehicles ##
    cartype_df = cal_countyvehicle_df.groupby(['Data Year', 'County', 'Fuel Category'])['Number of Vehicles'].sum().reset_index()  
    for year in cartype_df['Data Year'].unique():
        for county in cartype_df['County'].unique():
            # Get the subset of the dataframe for this year and county
            subset = cartype_df[(cartype_df['Data Year'] == year) & 
                                                       (cartype_df['County'] == county)]

            # Check if 'EV' exists in the 'Fuel Category' for this year and county
            if 'EV' not in subset['Fuel Category'].values:
                # If not, create a new row with 0 vehicles for 'EV'
                new_row_ev = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['EV'], 'Number of Vehicles': [0]})
                cartype_df = pd.concat([cartype_df, new_row_ev], ignore_index=True)

            # Check if 'Fossil Fuel' exists in the 'Fuel Category' for this year and county
            if 'Fossil Fuel' not in subset['Fuel Category'].values:
                # If not, create a new row with 0 vehicles for 'Fossil Fuel'
                new_row_fossil = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['Fossil Fuel'], 'Number of Vehicles': [0]})
                cartype_df = pd.concat([cartype_df, new_row_fossil], ignore_index=True)

    # Optionally, sort the dataframe by 'Data Year', 'County', and 'Fuel Category' for better readability
    cartype_df = cartype_df.sort_values(by=['Data Year', 'County', 'Fuel Category']).reset_index(drop=True)
    output_path = 'fueltype_per_countyyear_totals.csv'
    cartype_df.to_csv(output_path, index=False)
    return cartype_df
fueltype_per_county_totals =  fueltype_per_countyyear_totals_df('vehicle_data/california_vehicle.xlsx') 
fueltype_per_county_totals.head

<bound method NDFrame.head of       Data Year   County Fuel Category  Number of Vehicles
0          2010  Alameda            EV                  20
1          2010  Alameda   Fossil Fuel              885402
2          2010   Alpine            EV                   0
3          2010   Alpine   Fossil Fuel                1041
4          2010   Amador            EV                   1
...         ...      ...           ...                 ...
1647       2023  Ventura   Fossil Fuel              651851
1648       2023     Yolo            EV                6757
1649       2023     Yolo   Fossil Fuel              152560
1650       2023     Yuba            EV                 961
1651       2023     Yuba   Fossil Fuel               57567

[1652 rows x 4 columns]>

# I don't think we need to export it if it's all one file right? 

In [15]:
# # Now that our Primary Dataset is cleaned and maniputed, let's export it to a csv so that other people can use it 
# fueltype_per_countyyear_df.to_csv('./SIADS_Milestone_1/vehicle_df', index = False)

# Brett, I keep getting cartype as undefined. Can you clarify this code? I don't want to mess anything up on ya, Bud. 

In [16]:
# for year in cartype_df['Data Year'].unique():
#     for county in cartype_df['County'].unique():
#         # Get the subset of the dataframe for this year and county
#         subset = cartype_df[(cartype_df['Data Year'] == year) & 
#                                                    (cartype_df['County'] == county)]
#         
#         # Check if 'EV' exists in the 'Fuel Category' for this year and county
#         if 'EV' not in subset['Fuel Category'].values:
#             # If not, create a new row with 0 vehicles for 'EV'
#             new_row_ev = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['EV'], 'Number of Vehicles': [0]})
#             cartype_df = pd.concat([cartype_df, new_row_ev], ignore_index=True)
# 
#         # Check if 'Fossil Fuel' exists in the 'Fuel Category' for this year and county
#         if 'Fossil Fuel' not in subset['Fuel Category'].values:
#             # If not, create a new row with 0 vehicles for 'Fossil Fuel'
#             new_row_fossil = pd.DataFrame({'Data Year': [year], 'County': [county], 'Fuel Category': ['Fossil Fuel'], 'Number of Vehicles': [0]})
#             cartype_df = pd.concat([cartype_df, new_row_fossil], ignore_index=True)
# 
# # Optionally, sort the dataframe by 'Data Year', 'County', and 'Fuel Category' for better readability
# cartype_df = cartype_df.sort_values(by=['Data Year', 'County', 'Fuel Category']).reset_index(drop=True)
# 
# cartype_df.head()

In [17]:
## There are two variants of Out of State; let's filter those out since we are focusing on California Car Registration.  
in_state_only_filtered_df = fueltype_per_county_totals[~fueltype_per_county_totals['County'].isin(['Out of State','Out Of State'])]
in_state_only_filtered_df

,Data Year,County,Fuel Category,Number of Vehicles
0,2010,Alameda,EV,20
1,2010,Alameda,Fossil Fuel,885402
2,2010,Alpine,EV,0
3,2010,Alpine,Fossil Fuel,1041
4,2010,Amador,EV,1
...,...,...,...,...
1647,2023,Ventura,Fossil Fuel,651851
1648,2023,Yolo,EV,6757
1649,2023,Yolo,Fossil Fuel,152560
1650,2023,Yuba,EV,961


### Dataset 1: Exploratory Visual Plot of Vehicle Trends in California 
To plot vehicle buying trends in California, we need to do a little extra cleaning and manipulating of the dataset. 

In [18]:
# There is another unnecessary category, 'Other.' Let's explore its impact to determine if we should filter it or not. 
other_fuel_type_df = calcountyvehicle_df[(calcountyvehicle_df['Data Year'] == 2023) & 
                                  (calcountyvehicle_df['Dashboard Fuel Type Group'] == 'Other')]

# Calculate the sum of the 'Number of Vehicles' for the filtered data
total_vehicles_2023_other = other_fuel_type_df['Number of Vehicles'].sum()

# Print the result
print(f"The total number of vehicles in 2023 for 'Other' fuel type is: {total_vehicles_2023_other}")

The total number of vehicles in 2023 for 'Other' fuel type is: 8220


In [19]:
# We are going to assume that if there is a fuel typs that are not in fuel_types_1 they can be excluded. 
excluded_types = [fuel_type for fuel_type in fuel_types_2 
                  if fuel_type not in fuel_types_1]
print(f"The excluded fuel type categories are: {excluded_types}")

The excluded fuel type categories are: ['Flex Fuel', 'Natural Gas', 'Propane']


# Brett, i assumed county_df is the county column is the in_state_only_filtered_df['County'], is this correct? 
There is also an error with this code bit. I lost the plot after this point, can you finish describing your method? Thanks 

In [20]:
# county_df = in_state_only_filtered_df['County'].unique()
# # Get unique fuel types from both columns
# fuel_types_1 = county_df['Fuel Type'].unique()
# fuel_types_2 = calcountyvehicle_df['Dashboard Fuel Type Group'].unique()
# 
# # Create a DataFrame to display the values side by side
# fuel_types_df = pd.DataFrame({
#     'Fuel Type': pd.Series(fuel_types_1),
#     'Dashboard Fuel Type Group': pd.Series(fuel_types_2)
# })
# 
# # Display the DataFrame
# fuel_types_df

In [21]:
# cali_county_df = county_df[~county_df['County'].isin(['Out of State','Out Of State'])]
# cali_county_df['County'].unique()

In [22]:
# fuel_types = cali_county_df['Fuel Type'].unique()

In [23]:
# statewide_vehicle_totals = county_df.groupby(['Fuel Type', 'Data Year'])['Number of Vehicles'].sum().reset_index()
# statewide_vehicle_totals['Fuel Type'].unique()

In [24]:
# total_vehicles_by_year = county_df.groupby('Data Year')['Number of Vehicles'].sum().reset_index()

In [25]:
# # Finally, we can plot the data. 
# 
# plt.figure(figsize=(12, 6))
# 
# for fuel_type in fuel_types:
#     fuel_data = statewide_vehicle_totals[statewide_vehicle_totals['Fuel Type'] == fuel_type]
#     plt.plot(fuel_data['Data Year'], fuel_data['Number of Vehicles']/1000000, label=fuel_type)
# 
# plt.plot(total_vehicles_by_year['Data Year'], total_vehicles_by_year['Number of Vehicles']/1000000, 
#          label='Total Vehicles', color='black', linestyle='--', linewidth=2)
# 
# bev_data = statewide_vehicle_totals[statewide_vehicle_totals['Fuel Type'] == 'Battery Electric (BEV)']
# gasoline_data = statewide_vehicle_totals[statewide_vehicle_totals['Fuel Type'] == 'Gasoline']
# 
# merged_data = pd.merge(bev_data[['Data Year', 'Number of Vehicles']], gasoline_data[['Data Year', 'Number of Vehicles']],
#                        on='Data Year', suffixes=('_BEV', '_Gasoline'))
# 
# merged_data = pd.merge(merged_data, total_vehicles_by_year[['Data Year', 'Number of Vehicles']], 
#                        on='Data Year')
# #(merged_data.head())
# 
# merged_data['BEV (%)'] = (merged_data['Number of Vehicles_BEV'] / merged_data['Number of Vehicles']) * 100
# merged_data['Gasoline (%)'] = (merged_data['Number of Vehicles_Gasoline'] / merged_data['Number of Vehicles']) * 100
# 
# #column_labels = ['Year', 'BEV (%)', 'Gasoline (%)']
# plt.legend(title='Fuel Type')
# #plt.table(cellText=table_data, colLabels=column_labels, loc='right', cellLoc='center', bbox=[0.0, -0.5, 1, 0.3])
# ax1 = plt.gca()  # Get current axes instance for the main plot
# ax2 = ax1.twinx()  # Create a second axes sharing the same x-axis
# 
# 
# ax2.plot(merged_data['Data Year'], merged_data['Gasoline (%)'], color='red', linestyle='--', label='Gasoline (%)')
# ax2.set_ylabel('Gasoline Vehicles (%)', color='red')
# ax2.tick_params(axis='y', labelcolor='red')
# ax2.set_ylim(0, 100)
# ax2.set_ylabel('Percentage Total Cars')
# 
# plt.title('Total Vehicles by Fuel Type and Year')
# plt.xlabel('Year')
# ax1.set_ylabel('Number of Vehicles (millions)')
# plt.legend()
# 
# 
# plt.grid(True)
# plt.show()

In [26]:
# merged_data[['Data Year','BEV (%)','Gasoline (%)']].head(14)

In [27]:
# #table_to_be_merged=
# ## We want to take the data for vehicles by year and include only BEV vehicles for each county ##
# def bevpercentagevisual(year):
# 
#     bev_data_year = reduced_cali_county_df[(reduced_cali_county_df['Fuel Type'] == 'Battery Electric (BEV)') & 
#                                            (reduced_cali_county_df['Data Year'] == year)]
#     
#     
#     # Total vehicles for 2010 and 2023
#     total_vehicles_year = reduced_cali_county_df[reduced_cali_county_df['Data Year'] == year].groupby('County')['Number of Vehicles'].sum().reset_index()
#     
#     # Merge BEV data with total vehicles for each year
#     merged_year = pd.merge(bev_data_year.groupby('County')['Number of Vehicles'].sum().reset_index(), total_vehicles_year, on='County', suffixes=('_BEV', '_Total'))
#     
#     
#     # Calculate BEV percentage for 2010 and 2023
#     merged_year['BEV (%)'] = (merged_year['Number of Vehicles_BEV'] / merged_year['Number of Vehicles_Total']) * 100
#     
#     # Step 3: Plot the data side by side
#     fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(18, 10), sharey=True)
#     
#     # Plot 2010 data
#     axes.barh(merged_year['County'], merged_year['BEV (%)'], color='blue')
#     axes.set_xlabel('Percentage of BEVs (%)')
#     axes.set_ylabel('County')
#     axes.set_title('Percentage of BEVs by County (2010)')
#     
#     
#     
#     # Display the plots
#     plt.tight_layout()
#     plt.show()
# bevpercentagevisual(2013)


In [28]:
# # Filter data for 2010, 2018, and 2023
# years = [2010, 2018,2020, 2023]
# filtered_data = reduced_cali_county_df[reduced_cali_county_df['Data Year'].isin(years)]
# 
# total_vehicles = filtered_data.groupby(['Data Year', 'County'])['Number of Vehicles'].sum().reset_index()
# 
# bev_data = filtered_data[filtered_data['Fuel Type'] == 'Battery Electric (BEV)']
# bev_vehicles = bev_data.groupby(['Data Year', 'County'])['Number of Vehicles'].sum().reset_index()
# 
# merged_data = pd.merge(bev_vehicles, total_vehicles, on=['Data Year', 'County'], suffixes=('_BEV', '_Total'))
# 
# merged_data['BEV (%)'] = (merged_data['Number of Vehicles_BEV'] / merged_data['Number of Vehicles_Total']) * 100
# 
# 
# pivot_data = merged_data.pivot(index='County', columns='Data Year', values='BEV (%)').fillna(0)
# 
# 
# plt.figure(figsize=(12, 8))
# 
# 
# for year, color in zip(years, ['blue', 'orange', 'green','red']):
#     plt.barh(pivot_data.index, pivot_data[year], color=color, alpha=0.5, label=f'{year}')
# 
# 
# plt.xlabel('Percentage of BEVs (%)')
# plt.ylabel('County')
# plt.title('Percentage of Battery Electric Vehicles (BEVs) by County for 2010, 2018, and 2023')
# plt.legend(title='Year')
# 
# 
# plt.tight_layout()
# plt.show()

## Secondary Datasets: Cleaning and Manipulation 
The secondary datasets included PM 2.5 emissions by County and Population by County. 

In [29]:
# PM 2.5 emissions were recorded yearly and were recorded in separate files, which we uploaded to Github. Let's merge these files so that we can see Yearly County PM 2.5 emissions. 
csv_2010 = 'pm_data/ad_viz_plotval_data_2010.csv.crdownload.csv'
csv_2011 = 'pm_data/ad_viz_plotval_data_2011.csv.crdownload.csv'
csv_2012 = 'pm_data/ad_viz_plotval_data_2012.csv'
csv_2013 = 'pm_data/ad_viz_plotval_data_2013.csv'
csv_2014 = 'pm_data/ad_viz_plotval_data_2014.csv'
csv_2015 = 'pm_data/ad_viz_plotval_data_2015.csv'
csv_2016 = 'pm_data/ad_vis_plotval_data_2016.csv'
csv_2017 = 'pm_data/ad_viz_plotval_data_2017.csv'
csv_2018 = 'pm_data/ad_viz_plotval_data2018.csv'
csv_2019 = 'pm_data/ad_viz_plotval_data2019.csv'
csv_2020 = 'pm_data/ad_viz_plotval_data2020.csv'
csv_2021 = 'pm_data/ad_viz_plotval_data2020.csv'
csv_2022 = 'pm_data/ad_viz_plotval_data2022.csv'
csv_2023 = 'pm_data/ad_viz_plotval_data2023.csv'
csv_2024 = 'pm_data/ad_viz_plotval_data2024.csv'

In [30]:
# This function takes a file name and returns a mergeable dataframe. 
def avg_pm_value(filename): 
    # loading data
    df = pd.read_csv(filename)
    # We are only concerned with the following columns: 
    df = df[['Date', 'Daily Mean PM2.5 Concentration','Units', 'Site ID', 'County']]
    # changing date column to datetime 
    df['Date'] = pd.to_datetime(df['Date'])
    # adding year column 
    df['Data Year'] = df['Date'].dt.year
    # determining unique sites per county 
    unique_sites_per_county = df.groupby('County')['Site ID'].nunique().reset_index()
    unique_sites_per_county.rename(columns={'Site ID': 'Number of Site IDs/County'}, inplace=True) 
    # average pm2.5 values per county 
    avg_pm_value =df.groupby(['County', 'Data Year'])['Daily Mean PM2.5 Concentration'].mean().reset_index()
    avg_pm_value.rename(columns={'Daily Mean PM2.5 Concentration': 'Average PM 2.5/County'}, inplace=True)
    # merging tables 
    avg_df = unique_sites_per_county.merge(avg_pm_value, on='County')
    avg_df = avg_df.pivot_table(index ='County', values = ['Data Year','Average PM 2.5/County', 'Number of Site IDs/County'])
    avg_df.reset_index(inplace=True)
    return avg_df

In [31]:
# Putting all file names through avg_pm_value function 
df_2010 = avg_pm_value(csv_2010)
df_2011 = avg_pm_value(csv_2011)
df_2012 = avg_pm_value(csv_2012)
df_2013 = avg_pm_value(csv_2013)
df_2014 = avg_pm_value(csv_2014)
df_2015 = avg_pm_value(csv_2015)
df_2016 = avg_pm_value(csv_2016) 
df_2017 = avg_pm_value(csv_2017)
df_2018 = avg_pm_value(csv_2018)
df_2019 = avg_pm_value(csv_2019)
df_2020 = avg_pm_value(csv_2020)
df_2021 = avg_pm_value(csv_2021)
df_2022 = avg_pm_value(csv_2022)
df_2023 = avg_pm_value(csv_2023)
df_2024 = avg_pm_value(csv_2024)

In [32]:
# Creating one dataframe from all the Years. 
df_2010_2024= pd.concat([df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True, axis = 0)
df_2010_2024['Data Year'] = df_2010_2024['Data Year'].astype(int)
df_2010_2024 = df_2010_2024[['Data Year', 'County', 'Average PM 2.5/County', 'Number of Site IDs/County']]
# renaming dataset for clarity 
df_pm = df_2010_2024

## Now let's talk Population 
 We have two sources for population. One population is the total population, all ages included, and the second takes age into consideration. The first dataset we will look at here is the total population of Counties in California. ##The reason we wanted to see the total population was so that we could see how populated Counties were, which gives more prespective on why there are more or less cars in that area. 

In [33]:
# Our source dataset was seperated by decade, and we uploaded the corresponding datasets to GitHub (2010-2023). The dataset had multiple sheets, but we were interested in the sheet that recorded the population by county ('Table 1 County State'). We will import those datasets that here. 
df_2010_2020 = pd.read_excel('./E-4_2010-2020-Internet-Version.xlsx', sheet_name= 'Table 1 County State')
df_2021_2023 = pd.read_excel('./E-4_2021_2024_InternetVersion (1).xlsx',sheet_name='Table 1 County State')

In [34]:
# Let's clean up df_2010_2020 first. 
# renaming unnamed columns to correlated year
mapping = {
    'Table 1: E-4 Population Estimates for Counties and State 2011-2020 with 2010 Benchmark': 'County',
    'Unnamed: 1': '2010',
    'Unnamed: 2': '2011',
    'Unnamed: 3': '2012',
    'Unnamed: 4': '2013',
    'Unnamed: 5': '2014',
    'Unnamed: 6': '2015',
    'Unnamed: 7': '2016',
    'Unnamed: 8': '2017',
    'Unnamed: 9': '2018',
    'Unnamed: 10': '2019',
    'Unnamed: 11': '2020'
}
df_2010_2020.rename(columns=mapping, inplace=True)
# Removing Unnessary Columns 
df_2010_2020 = df_2010_2020.drop(columns=['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'])
# Removing 'State Total' from 'County' and redundant row
df_2010_2020 = df_2010_2020[~df_2010_2020['County'].isin(['State Total', 'COUNTY'])]
df_2010_2020.reset_index(drop=True, inplace=True)

In [35]:
# Now let's clean up the second dataset 
# renaming unnamed columns to correlated year
mapping_2 = {
    'About the Data': 'County',
    'Unnamed: 2': '2021',
    'Unnamed: 3': '2022',
    'Unnamed: 4': '2023',
    'Unnamed: 5': '2024'
}
df_2021_2023.rename(columns=mapping_2, inplace=True)
df_2021_2023 = df_2021_2023.drop(columns=['Unnamed: 1'])
# Removing 'State Total' from 'County' and redundant rows
df_2021_2023 = df_2021_2023[~df_2021_2023['County'].isin(['State Total', 'COUNTY', 'Table 1: E-4 Population Estimates for Counties and State 2021-2024 with 2020 Benchmark'])]
df_2021_2023.reset_index(drop=True, inplace=True)

In [36]:
# merging the two datasets to create one with the population from 2010-2023. 
total_pop_df = pd.merge(df_2010_2020, df_2021_2023, on=['County'], how='outer')

In [37]:
# pivoting table so that Counties make up the rows and Years the columns
df_pop = pd.melt(total_pop_df, id_vars=['County'], var_name='Year', value_name='Population')

df_pop['Year'] = df_pop['Year'].str.replace(":", "", regex=False)
df_pop.rename(columns={'Year': 'Data Year'}, inplace=True) 
# converting the 'Population' column to numeric 
df_pop['Population'] = pd.to_numeric(df_pop['Population'], errors='coerce')
# calculating the Z-score for the 'Population' column
df_pop['Z Score'] = zscore(df_pop['Population'])
# formatting Data Year to integer  
df_pop['Data Year'] = df_pop['Data Year'].astype(int)

## Aged Population
Now let's look at the dataset that takes age into consideration. The reason we wanted to also take age into consideration is that most people under the age of 18 do not add a car, so the total population dataset doesn't accurately represent car buying trends. 

In [38]:
# This function takes a dataset and its corresponding year and returns a mergeable dataframe. Again, our source dataset was seperated by decade, and we uploaded the corresponding datasets to GitHub (2010-2023).

def aged_population(dataset, datayear):
    # load files 
    df = pd.read_csv(dataset)
    df = df[df['Label (Grouping)'].str.strip() == '18 years and over']
    df = df[df['Label (Grouping)'].str.contains('18 years and over', case=False, na=False)]
    # extract unique county names
    county_names = [col.split("!!")[0] for col in df.columns if "California" in col]
    county_names = list(set(county_names))
    # extract county total and margin of error from main dataset
    county_total = [f"{county}!!Total!!Estimate" for county in county_names]
    county_margin_of_error = [f"{county}!!Total!!Margin of Error" for county in county_names]
    # making df of those
    county_total = df.loc[:, county_total]
    county_margin_of_error = df.loc[:, county_margin_of_error]
    # creating county total population estimate df 
    ct_df =county_total.transpose()
    ct_df = ct_df.reset_index() 
    # some years record >21, others >18. 
    ct_df.columns = ['County' , 'Population Estimate, >18']
    ct_df['County'] = ct_df['County'].str.split(' County').str[0]
    ct_df = ct_df.sort_values(by='County', ascending=True).reset_index(drop=True)
    # creating county margin of error 
    me_df = county_margin_of_error.transpose()
    me_df = me_df.reset_index() 
    me_df.columns = ['County' , 'Population Margin of Error, >18']
    me_df['County'] = me_df['County'].str.split(' County').str[0]
    me_df = me_df.sort_values(by='County', ascending=True).reset_index(drop=True) 
    # merging on county 
    ct_me_df = pd.merge(ct_df, me_df, how='inner', on = ['County'])
    # creating Data Year column to datetime 
    ct_me_df['Data Year'] = datayear
    ct_me_df['Data Year'] = ct_me_df['Data Year'].astype(int)
    return ct_me_df

In [39]:
# run function with dataset files and the corresponding year. 
totpop2010 = aged_population('pm_data/Pop_Over_21_2010.S0101-2024-09-27T210322.csv', 2010)
totpop2011 = aged_population('pm_data/Pop_Over_21_2011.S0101-2024-09-27T210305.csv', 2011)
totpop2012 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2012)
totpop2013 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2013)
totpop2014 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2014)
totpop2015 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2015)
totpop2016 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2016)
totpop2017 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2017)
totpop2018 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2018)
totpop2019 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2019)
totpop2020 = aged_population('pm_data/pop_over_18_2020.csv', 2020)
totpop2021 = aged_population('pm_data/Pop_Over_21_2012.S0101-2024-09-27T210137.csv', 2021)
totpop2022 = aged_population('pm_data/Pop_Over_21_2022.csv', 2022)
totpop2023 = aged_population('pm_data/Pop_Over_21_2023.S0101-2024-09-27T205616.csv', 2023)
# stacking totpop
dfs = [totpop2010, totpop2011, totpop2012, totpop2013, totpop2014, totpop2015, totpop2016, totpop2017, totpop2018, totpop2019, totpop2021, totpop2022, totpop2023]
aged_pop_2010_2023= pd.concat(dfs, ignore_index= True)
aged_pop_2010_2023

,County,"Population Estimate, >18","Population Margin of Error, >18",Data Year
0,Alameda,77.5%,*****,2010
1,Butte,79.3%,±0.3,2010
2,Contra Costa,75.2%,±0.1,2010
3,El Dorado,77.2%,±0.2,2010
4,Fresno,70.1%,±0.1,2010
...,...,...,...,...
535,Tehama,"49,287",*****,2023
536,Tulare,"338,510",±144,2023
537,Ventura,"651,462",*****,2023
538,Yolo,"178,433",±709,2023


# Merging to create Main Dataframe

In [40]:
# Let's select only Electric Vehicles to see if higher EV values result in lower PM 2.5 Emissions 
df_ev_vehicles = fueltype_per_county_totals[fueltype_per_county_totals['Fuel Category'] == 'EV']
df_ev_vehicles = df_ev_vehicles.rename(columns={'Number of Vehicles': 'Number of EV'})
df_ev_vehicles = df_ev_vehicles.drop(columns=['Fuel Category'])
df_ev_vehicles

,Data Year,County,Number of EV
0,2010,Alameda,20
2,2010,Alpine,0
4,2010,Amador,1
6,2010,Butte,0
8,2010,Calaveras,0
...,...,...,...
1642,2023,Tulare,3837
1644,2023,Tuolumne,698
1646,2023,Ventura,31980
1648,2023,Yolo,6757


In [41]:
# Let's see if there are any differences between df_ev_vehicles and 
# strip df_pm of spaces 
df_pm['County'] = df_pm['County'].str.strip()
counties = pd.DataFrame({
    'County_df_vehicles': df_ev_vehicles['County'],
    'County_df_pm': df_pm['County']
})
counties.head()
# Get unique counties from both DataFrames
counties_vehicles = set(df_ev_vehicles['County'])
counties_pm = set(df_pm['County'])

# Find differences
only_in_vehicles = counties_vehicles - counties_pm
only_in_pm = counties_pm - counties_vehicles

print("Only in df_ev_vehicles:", only_in_vehicles)
print("Only in df_pm:", only_in_pm)


Only in df_ev_vehicles: {'Out Of State', 'Modoc', 'Sierra', 'Yuba', 'Amador', 'Lassen', 'Tuolumne'}
Only in df_pm: set()


In [42]:
# Since df_pm has higher requirements for taking environmental data, we will shape based on df_pm. 
df_pm_vehicles = pd.merge(df_pm, df_ev_vehicles, on=['Data Year', 'County'], how='inner')

In [43]:
# Now let's merge population to df_pm_vehicles 
# clean df_pop county column to match df_pm_vehicle county column 
df_pop['County'] = df_pop['County'].str.strip()
df_pop['County'] = df_pop['County'].str.title()
df_pop['County'] = df_pop['County'].str.replace(r'\s+', ' ', regex=True) 

In [44]:
df_pop['Data Year']

0      2010
1      2010
2      2010
3      2010
4      2010
       ... 
865    2024
866    2024
867    2024
868    2024
869    2024
Name: Data Year, Length: 870, dtype: int64

In [45]:
# Get unique counties from both DataFrames
counties_pop = set(df_pop['County'])
counties_pm_vehicles = set(df_pm_vehicles['County'])

# Find differences
only_in_pop = counties_pop - counties_pm_vehicles
only_in_pm_vehicles = counties_pm_vehicles - counties_pop

print("Only in df_pop:", only_in_pop)
print("Only in df_pm_vehicles:", only_in_pm_vehicles)
# for now, we will merge on left for df_pm_vehicles 

Only in df_pop: {'Modoc', 'Sierra', 'Yuba', 'Amador', 'Lassen', 'Tuolumne'}
Only in df_pm_vehicles: set()


In [46]:
# Let's merge df_pm_vehicles and total population 
df_pm_vehicles_pop= pd.merge(df_pm_vehicles, df_pop, on=['Data Year', 'County'], how='inner')
df_pm_vehicles_pop.rename(columns={'Z Score_x':'Total Population Z Score'}, inplace=True)
df_pm_vehicles_pop.rename(columns={'Population_x':'Total Population'}, inplace=True)
# renaming df_pm_vehicles_pop to main_df for clarity 
main_df = df_pm_vehicles_pop

In [47]:
 # Let's see if there are any main differences between the Counties in the total population dataframe and the aged population dataframe. 
aged_pop_2010_2023['County'] = aged_pop_2010_2023['County'].str.strip()
counties = pd.DataFrame({
    'County_main_df': df_pop['County'],
    'County_aged_pop_2010_2023': aged_pop_2010_2023['County']
})
counties.head()
# Get unique counties from both DataFrames
pop_df_co = set(df_pop['County'])
aged_pop_2010_2023_co = set(aged_pop_2010_2023['County'])

# Find differences
only_in_pop_df_co = pop_df_co - aged_pop_2010_2023_co
only_in_aged_pop_2010_2023_co = aged_pop_2010_2023_co - pop_df_co

print("Only in pop_df:", only_in_pop_df_co)
print("Only in aged_pop_2010_2023:", only_in_aged_pop_2010_2023_co)

Only in pop_df: set()
Only in aged_pop_2010_2023: set()


In [48]:
# merging aged_pop_2010_2023 to main_df 
main_df = pd.merge(main_df, aged_pop_2010_2023, how='inner', on = ['Data Year', 'County'])
main_df

,Data Year,County,Average PM 2.5/County,Number of Site IDs/County,Number of EV,Population,Z Score,"Population Estimate, >18","Population Margin of Error, >18"
0,2010,Alameda,8.165623,5.0,20,1510271,0.585020,77.5%,*****
1,2010,Butte,8.776451,3.0,0,220000,-0.311810,79.3%,±0.3
2,2010,Contra Costa,7.606920,1.0,10,1049025,0.264421,75.2%,±0.1
3,2010,El Dorado,2.195902,1.0,0,181058,-0.338878,77.2%,±0.2
4,2010,Fresno,11.729538,6.0,2,930450,0.182003,70.1%,±0.1
...,...,...,...,...,...,...,...,...,...
475,2023,Sutter,8.442737,1.0,1147,98248,-0.396437,"74,369",±180
476,2023,Tehama,6.108611,1.0,398,64710,-0.419748,"49,287",*****
477,2023,Tulare,10.567731,4.0,3837,474680,-0.134790,"338,510",±144
478,2023,Ventura,5.870643,5.0,31980,825960,0.109375,"651,462",*****


In [50]:
# there is some percentages and whole numbers, let's make them all whole numbers 

def convert_to_population_estimate(row):
    # converting percentage to decimal, if applicable 
    if isinstance(row['Population Estimate, >18'], str) and '%' in row['Population Estimate, >18']:
        percent_value = float(row['Population Estimate, >18'].strip('%')) / 100
        return percent_value * row['Population']
    else:
        return row['Population Estimate, >18']

# apply to update 'Total Population Estimate, >21'
main_df['Population Estimate, >18'] = main_df.apply(convert_to_population_estimate, axis=1)

main_df 

,Data Year,County,Average PM 2.5/County,Number of Site IDs/County,Number of EV,Population,Z Score,"Population Estimate, >18","Population Margin of Error, >18"
0,2010,Alameda,8.165623,5.0,20,1510271,0.585020,1170460.025,*****
1,2010,Butte,8.776451,3.0,0,220000,-0.311810,174460.0,±0.3
2,2010,Contra Costa,7.606920,1.0,10,1049025,0.264421,788866.8,±0.1
3,2010,El Dorado,2.195902,1.0,0,181058,-0.338878,139776.776,±0.2
4,2010,Fresno,11.729538,6.0,2,930450,0.182003,652245.45,±0.1
...,...,...,...,...,...,...,...,...,...
475,2023,Sutter,8.442737,1.0,1147,98248,-0.396437,"74,369",±180
476,2023,Tehama,6.108611,1.0,398,64710,-0.419748,"49,287",*****
477,2023,Tulare,10.567731,4.0,3837,474680,-0.134790,"338,510",±144
478,2023,Ventura,5.870643,5.0,31980,825960,0.109375,"651,462",*****
